In [1]:
import pandas as pd

In [30]:
# PATH = '../table/detect_seq.StatsTest.table_CTRL-VEGFA-Vector-PD_TREAT-GBEmini-AP-RNF2-All-PD_rep1.filtered_SMN-4_SCN-10_SMR-0.1_RPN-1_RTN-2.C2T_merge_G2A.tsv'
PATH = '../table/detect_seq.StatsTest.table_CTRL-VEGFA-Vector-PD_TREAT-GBEmini-dU-RNF2-All-PD_rep1.filtered_SMN-4_SCN-10_SMR-0.1_RPN-1_RTN-2.C2T_merge_G2A.tsv'
df = pd.read_csv(PATH,sep='\t')
print(df.shape)
df.head(2)

(18978, 17)


,chr_name,region_start,region_end,ctrl_count,treat_count,ctrl_mut_count,treat_mut_count,ctrl_count.norm,treat_count.norm,ctrl_mut_count.norm,treat_mut_count.norm,count_info,log2_FC,log2_FC_mut,test_state,p_value,FDR
0,chr1,187019,187019,18,16,0,0,0.129787,0.148238,0.000000,0.000000,"0,1 18,0 16,0",0.191765,NaN,NoTest,NaN,NaN
1,chr1,265421,265423,16,26,2,7,0.115366,0.240886,0.014421,0.064854,"0,1,2 14,0,2 19,3,4",1.062130,2.169045,TestOK,0.082913,0.306784


In [31]:
# {if($NF<0.05 && $13>1 && $10<3) print $0}
df_select = df[(df.p_value <= 0.005) & (df['ctrl_mut_count.norm'] < 3) & (df['log2_FC'] > 1)]
print(df_select.shape)
df_select.head(2)

(38, 17)


,chr_name,region_start,region_end,ctrl_count,treat_count,ctrl_mut_count,treat_mut_count,ctrl_count.norm,treat_count.norm,ctrl_mut_count.norm,treat_mut_count.norm,count_info,log2_FC,log2_FC_mut,test_state,p_value,FDR
338,chr1,116894504,116894504,11,18,3,13,0.079314,0.166767,0.021631,0.120443,"0,1 8,3 5,13",1.072184,2.477168,TestOK,0.003292,0.306784
432,chr1,124919388,124919388,9,20,3,14,0.064894,0.185297,0.021631,0.129708,"0,1 6,3 6,14",1.513693,2.584083,TestOK,0.001824,0.285094


In [32]:
df_bed = df_select[['chr_name','region_start','region_end']]
df_bed['middle'] = ((df_bed['region_start'] + df_bed['region_end'])/2).map(int)
df_bed.head(2)

<ipython-input-32-3ac48ad4d1a2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bed['middle'] = ((df_bed['region_start'] + df_bed['region_end'])/2).map(int)


,chr_name,region_start,region_end,middle
338,chr1,116894504,116894504,116894504
432,chr1,124919388,124919388,124919388


In [33]:
df_bed['tss'] = df_bed['middle'] - 20
df_bed['tes'] = df_bed['middle'] + 20
df_bed.head()

<ipython-input-33-ba1481cf0ee6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bed['tss'] = df_bed['middle'] - 20
<ipython-input-33-ba1481cf0ee6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bed['tes'] = df_bed['middle'] + 20


,chr_name,region_start,region_end,middle,tss,tes
338,chr1,116894504,116894504,116894504,116894484,116894524
432,chr1,124919388,124919388,124919388,124919368,124919408
1660,chr2,94557172,94557172,94557172,94557152,94557192
1680,chr2,99224285,99224285,99224285,99224265,99224305
1779,chr3,16812877,16812877,16812877,16812857,16812897


In [34]:
df_bed = df_bed[['chr_name','tss','tes']]
df_bed.head(2)

,chr_name,tss,tes
338,chr1,116894484,116894524
432,chr1,124919368,124919408


In [36]:
print(PATH)

../table/detect_seq.StatsTest.table_CTRL-VEGFA-Vector-PD_TREAT-GBEmini-dU-RNF2-All-PD_rep1.filtered_SMN-4_SCN-10_SMR-0.1_RPN-1_RTN-2.C2T_merge_G2A.tsv


In [37]:
# attention! local path!
path_out = '/Users/mac/mac_data/3.projects/21.2020-11_1109_GBE_DETECT-Seq_new_protocol/igv-snapshot'
#
treat = 'GBEmini-dU-RNF2-All-PD_rep1'
ctrl = 'VEGFA-Vector-PD_rep1'


text = "maxPanelHeight 1500\nsnapshotDirectory {path}\n".format(path=path_out)

for chr_, tss, tes in df_bed.values.tolist():
    path_out_png = '293T-bat_TREAT-{treat}_CTRL-{ctrl}_{chr}_{tss}_{tes}.snapshot.png'.format(treat=treat,ctrl=ctrl,chr=chr_,tss=tss,tes=tes)
    text +="goto {chr}:{tss}-{tes}\nsnapshot {path_out_png}\n".format(chr=chr_,tss=tss,tes=tes,path_out_png=path_out_png)
print('\n'.join(text.split('\n')))
print('...\n')



path_out_batch = PATH.replace('tsv','snapshot_script.txt')
with open(path_out_batch,'w') as f:
    f.write(text)

maxPanelHeight 1500
snapshotDirectory /Users/mac/mac_data/3.projects/21.2020-11_1109_GBE_DETECT-Seq_new_protocol/igv-snapshot
goto chr1:116894484-116894524
snapshot 293T-bat_TREAT-GBEmini-dU-RNF2-All-PD_rep1_CTRL-VEGFA-Vector-PD_rep1_chr1_116894484_116894524.snapshot.png
goto chr1:124919368-124919408
snapshot 293T-bat_TREAT-GBEmini-dU-RNF2-All-PD_rep1_CTRL-VEGFA-Vector-PD_rep1_chr1_124919368_124919408.snapshot.png
goto chr2:94557152-94557192
snapshot 293T-bat_TREAT-GBEmini-dU-RNF2-All-PD_rep1_CTRL-VEGFA-Vector-PD_rep1_chr2_94557152_94557192.snapshot.png
goto chr2:99224265-99224305
snapshot 293T-bat_TREAT-GBEmini-dU-RNF2-All-PD_rep1_CTRL-VEGFA-Vector-PD_rep1_chr2_99224265_99224305.snapshot.png
goto chr3:16812857-16812897
snapshot 293T-bat_TREAT-GBEmini-dU-RNF2-All-PD_rep1_CTRL-VEGFA-Vector-PD_rep1_chr3_16812857_16812897.snapshot.png
goto chr4:50304977-50305017
snapshot 293T-bat_TREAT-GBEmini-dU-RNF2-All-PD_rep1_CTRL-VEGFA-Vector-PD_rep1_chr4_50304977_50305017.snapshot.png
goto chr4:5055